# Import libraries

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch

# Set the random seed
np.random.seed(1234)

# Set the device into GPU

In [3]:
if torch.cuda.is_available():
  device = torch.device("cuda")
print(f"Device using: {device}")

Device using: cuda


# Install the ucimlrepo package

In [7]:
# pip install ucimlrepo

# Load the dataframe 1 (df1)

*   Shape of X: (2278, 7)
*   Shape of y: (2278, 1)



In [46]:
# Dataset 1: NHANES age prediction.csv
#(National Health and Nutrition Health Sur- vey 2013-2014 (NHANES) Age Prediction Subset)
df1 = pd.read_csv('./dataset/NHANES_age_prediction.csv')
df1

FileNotFoundError: [Errno 2] No such file or directory: './dataset/NHANES_age_prediction.csv'

# Load the dataset 2 (df2)
Please uncomment it to use this dataset

*   Shape of X: (699, 9)
*   Shape of y: (699, 1)

In [ ]:
# from ucimlrepo import fetch_ucirepo

# # fetch dataset
# dataset2 = fetch_ucirepo(id=15)

# df2 = dataset2.data

# # data (as pandas dataframes)
# X = dataset2.data.features
# y = dataset2.data.targets

# # metadata
# print(dataset2.metadata)

# # variable information
# print(dataset2.variables)

In [26]:
print(X)

     Clump_thickness  Uniformity_of_cell_size  Uniformity_of_cell_shape  \
0                  5                        1                         1   
1                  5                        4                         4   
2                  3                        1                         1   
3                  6                        8                         8   
4                  4                        1                         1   
..               ...                      ...                       ...   
694                3                        1                         1   
695                2                        1                         1   
696                5                       10                        10   
697                4                        8                         6   
698                4                        8                         8   

     Marginal_adhesion  Single_epithelial_cell_size  Bare_nuclei  \
0                    1         

# Preprocess the dataset 1


In [39]:
df1.info()
# Clean the dataset
df1.isnull().sum()

# #print the feature shape and classes of dataset
# (N,D), C = X.shape, np.unique(y)
# print(f'instances (N) \t {N} \n features (D) \t {D} \n classes (C) \t {C}')


# #generates an indices array from 0 to N-1 and permutes it
# inds = np.random.permutation(N)


# #split the dataset into train and test
# x_train, y_train = X[inds[:100]], y[inds[:100]]
# x_test, y_test = X[inds[100:]], y[inds[100:]]

# #visualization of the data
# plt.scatter(x_train[:,0], x_train[:,1], c=y_train, marker='o', label='train')
# plt.scatter(x_test[:,0], x_test[:,1], c=y_test, marker='s', label='test')
# plt.legend()
# plt.ylabel('sepal length')
# plt.xlabel('sepal width')
# plt.show()

TypeError: 'NoneType' object is not callable